In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
conn = sqlite3.connect('parch-and-posey.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [5]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,7,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,222,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,223,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,583,"CREATE TABLE accounts (\tid integer,\tname bpc..."


Task 1:

Run the query ebtered below to notice the row with missing data.

In [8]:
pd.read_sql_query(sql='''
SELECT *
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id
WHERE o.total IS NULL;
''', con=conn)

,id,name,website,lat,long,primary_poc,sales_rep_id,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1731,Goldman Sachs Group,www.gs.com,40.757444,-73.967309,Loris Manfredi,321690,None,None,None,None,None,None,None,None,None,None,None


Task 2:

Use COALESCE to fill in the accounts.id column with the account.id for the NULL value for the table in 1.

In [14]:
pd.read_sql_query(sql='''
SELECT COALESCE (account_id, 1) filled_id, *
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id
WHERE o.total IS NULL;
''', con=conn)

,filled_id,id,name,website,lat,long,primary_poc,sales_rep_id,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1,1731,Goldman Sachs Group,www.gs.com,40.757444,-73.967309,Loris Manfredi,321690,None,None,None,None,None,None,None,None,None,None,None


Solution:

SELECT COALESCE(a.id, a.id) filled_id, a.name, a.website, a.lat, a.long, a.primary_poc, a.sales_rep_id, o.*
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id
WHERE o.total IS NULL;

Task 3:

Use COALESCE to fill in the orders.account_id column with the account.id for the NULL value for the table in 1.

In [16]:
pd.read_sql_query(sql='''
SELECT COALESCE (account_id, a.id) filled_id, *
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id
WHERE o.total IS NULL;
''', con=conn)

,filled_id,id,name,website,lat,long,primary_poc,sales_rep_id,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1731,1731,Goldman Sachs Group,www.gs.com,40.757444,-73.967309,Loris Manfredi,321690,None,None,None,None,None,None,None,None,None,None,None


Solution:

SELECT COALESCE(a.id, a.id) filled_id, a.name, a.website, a.lat, a.long, a.primary_poc, a.sales_rep_id, COALESCE(o.account_id, a.id) account_id, o.occurred_at, o.standard_qty, o.gloss_qty, o.poster_qty, o.total, o.standard_amt_usd, o.gloss_amt_usd, o.poster_amt_usd, o.total_amt_usd
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id
WHERE o.total IS NULL;

Task 4:

Use COALESCE to fill in each of the qty and usd columns with 0 for the table in 1.

In [27]:
pd.read_sql_query(sql='''
SELECT a.*, o.id, COALESCE (account_id, a.id) account_id, occurred_at,
COALESCE(standard_qty, 0) standard_qty, COALESCE(gloss_qty, 0) gloss_qty, COALESCE(poster_qty, 0) poster_qty,
COALESCE(total, 0) total, COALESCE(standard_amt_usd, 0) standard_amt_usd, COALESCE(gloss_amt_usd, 0) gloss_amt_usd,
COALESCE(poster_amt_usd, 0) poster_amt_usd, COALESCE(total_amt_usd, 0) total_amt_usd
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id
WHERE o.total IS NULL;
''', con=conn)

,id,name,website,lat,long,primary_poc,sales_rep_id,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1731,Goldman Sachs Group,www.gs.com,40.757444,-73.967309,Loris Manfredi,321690,None,1731,None,0,0,0,0,0,0,0,0


Solution:

SELECT COALESCE(a.id, a.id) filled_id, a.name, a.website, a.lat, a.long, a.primary_poc, a.sales_rep_id, COALESCE(o.account_id, a.id) account_id, o.occurred_at, COALESCE(o.standard_qty, 0) standard_qty, COALESCE(o.gloss_qty,0) gloss_qty, COALESCE(o.poster_qty,0) poster_qty, COALESCE(o.total,0) total, COALESCE(o.standard_amt_usd,0) standard_amt_usd, COALESCE(o.gloss_amt_usd,0) gloss_amt_usd, COALESCE(o.poster_amt_usd,0) poster_amt_usd, COALESCE(o.total_amt_usd,0) total_amt_usd
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id
WHERE o.total IS NULL;

Task 5:

Run the query in 1 with the WHERE removed and COUNT the number of id S.

In [20]:
# BEWARE!

pd.read_sql_query(sql='''
SELECT *
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id = 1731;
''', con=conn)

,id,name,website,lat,long,primary_poc,sales_rep_id,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1001,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500,None,None,None,None,None,None,None,None,None,None,None
1,1011,Exxon Mobil,www.exxonmobil.com,41.169156,-73.849374,Sung Shields,321510,None,None,None,None,None,None,None,None,None,None,None
2,1021,Apple,www.apple.com,42.290495,-76.084009,Jodee Lupo,321520,None,None,None,None,None,None,None,None,None,None,None
3,1031,Berkshire Hathaway,www.berkshirehathaway.com,40.949021,-75.763898,Serafina Banda,321530,None,None,None,None,None,None,None,None,None,None,None
4,1041,McKesson,www.mckesson.com,42.217093,-75.284998,Angeles Crusoe,321540,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,4461,KKR,www.kkr.com,45.545353,-122.655247,Buffy Azure,321970,None,None,None,None,None,None,None,None,None,None,None
347,4471,Oneok,www.oneok.com,45.513513,-122.681500,Esta Engelhardt,321960,None,None,None,None,None,None,None,None,None,None,None
348,4481,Newmont Mining,www.newmont.com,45.494117,-122.669460,Khadijah Riemann,321970,None,None,None,None,None,None,None,None,None,None,None
349,4491,PPL,www.pplweb.com,45.491720,-122.671880,Deanne Hertlein,321960,None,None,None,None,None,None,None,None,None,None,None


In [34]:
%%timeit

pd.read_sql_query(sql='''
SELECT *
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id
WHERE a.id = 1731;
''', con=conn)

10.1 ms ± 597 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [35]:
%%timeit

pd.read_sql_query(sql='''
WITH a AS (SELECT *
FROM accounts 
WHERE id = 1731)

SELECT *
FROM a
LEFT JOIN orders o
ON a.id = o.account_id

''', con=conn)

9.95 ms ± 481 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [36]:
# my solution of Task 5

pd.read_sql_query(sql='''
SELECT COUNT(*) count_with_NULLs
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id;
''', con=conn)

,count_with_NULLs
0,6913


In [37]:
# BAWARE / NULL not COUNTed

pd.read_sql_query(sql='''
SELECT COUNT(o.id) count_without_NULLs
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id;
''', con=conn)

,count_without_NULLs
0,6912


Solution:

SELECT COUNT(*)
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id;

Task 6:

Run the query in 5, but with the COALESCE function used in questions 2 thru 4.

In [50]:
pd.read_sql_query(sql='''
SELECT COUNT(o.account_id) count_without_NULLs
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id;
''', con=conn)

,count_without_NULLs
0,6912


In [51]:
pd.read_sql_query(sql='''
SELECT COUNT(COALESCE(account_id, a.id)) count_with_NULLs
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id;
''', con=conn)

,count_with_NULLs
0,6913


In [52]:
pd.read_sql_query(sql='''
SELECT COUNT(account_id) count_with_NULLs
FROM (SELECT a.*, o.id, COALESCE (account_id, a.id) account_id, occurred_at,
COALESCE(standard_qty, 0) standard_qty, COALESCE(gloss_qty, 0) gloss_qty, COALESCE(poster_qty, 0) poster_qty,
COALESCE(total, 0) total, COALESCE(standard_amt_usd, 0) standard_amt_usd, COALESCE(gloss_amt_usd, 0) gloss_amt_usd,
COALESCE(poster_amt_usd, 0) poster_amt_usd, COALESCE(total_amt_usd, 0) total_amt_usd
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id) t1;
''', con=conn)

,count_with_NULLs
0,6913


Solution:

SELECT COALESCE(a.id, a.id) filled_id, a.name, a.website, a.lat, a.long, a.primary_poc, a.sales_rep_id, COALESCE(o.account_id, a.id) account_id, o.occurred_at, COALESCE(o.standard_qty, 0) standard_qty, COALESCE(o.gloss_qty,0) gloss_qty, COALESCE(o.poster_qty,0) poster_qty, COALESCE(o.total,0) total, COALESCE(o.standard_amt_usd,0) standard_amt_usd, COALESCE(o.gloss_amt_usd,0) gloss_amt_usd, COALESCE(o.poster_amt_usd,0) poster_amt_usd, COALESCE(o.total_amt_usd,0) total_amt_usd
FROM accounts a
LEFT JOIN orders o
ON a.id = o.account_id;